# 6강) FAISS 실습

### Requirements

In [ ]:
# > /dev/null 2>&1 # execute command in silence
!pip install datasets==1.4.1 > /dev/null 2>&1 # execute command in silence
!pip install transformers==4.4.1 > /dev/null 2>&1
!pip install tqdm==4.41.1 > /dev/null 2>&1
!pip install faiss-cpu > /dev/null 2>&1

In [ ]:
import torch
import random
from pprint import pprint

In [ ]:
# 1. 미리 학습해둔 encoder file 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eMfMzv0gkcTSBQAxtQrFMP_pC5sEeQQq' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1eMfMzv0gkcTSBQAxtQrFMP_pC5sEeQQq" -O dense_encoder.tar.gz && rm -rf /tmp/cookies.txt

# 2. .tar.gz file 압축해제 
!tar -xf ./dense_encoder.tar.gz

--2021-05-09 22:55:04--  https://docs.google.com/uc?export=download&confirm=bEXp&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF
Resolving docs.google.com (docs.google.com)... 74.125.195.101, 74.125.195.113, 74.125.195.138, ...
Connecting to docs.google.com (docs.google.com)|74.125.195.101|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://doc-04-50-docs.googleusercontent.com/docs/securesc/vboqok24tal3g5ioup05nrm566vcm438/2v0erhs5hldueepcd5n4vsjpfutljdim/1620600900000/10000090644219567406/12388360935774394911Z/1eD57pCCvhiXLrHBb1j07QX1VQQDogejF?e=download [following]
--2021-05-09 22:55:05--  https://doc-04-50-docs.googleusercontent.com/docs/securesc/vboqok24tal3g5ioup05nrm566vcm438/2v0erhs5hldueepcd5n4vsjpfutljdim/1620600900000/10000090644219567406/12388360935774394911Z/1eD57pCCvhiXLrHBb1j07QX1VQQDogejF?e=download
Resolving doc-04-50-docs.googleusercontent.com (doc-04-50-docs.googleusercontent.com)... 74.125.20.132, 2607:f8b0:400e:c07::84
Connecting 

## [과제 1] Passage retrieval 준비하기

In [ ]:
from transformers import BertModel, BertPreTrainedModel, BertConfig, AutoTokenizer

class BertEncoder(BertPreTrainedModel):
  def __init__(self, config):
    super(BertEncoder, self).__init__(config)

    self.bert = BertModel(config)
    self.init_weights()
      
  def forward(self, input_ids, 
              attention_mask=None, token_type_ids=None): 
  
      outputs = self.bert(input_ids,
                          attention_mask=attention_mask,
                          token_type_ids=token_type_ids)
      
      pooled_output = outputs[1]

      return pooled_output

# TODO: load a model trained at Daily Mission 4

model_checkpoint = "bert-base-multilingual-cased"
p_encoder = BertEncoder.from_pretrained(model_checkpoint).cuda()
q_encoder = BertEncoder.from_pretrained(model_checkpoint).cuda()
model_dict = torch.load("./dense_encoder/encoder.pth")
p_encoder.load_state_dict(model_dict['p_encoder'])
q_encoder.load_state_dict(model_dict['q_encoder'])

# You can use "bert-base-multilingual-cased" for tokenizer, if you did not save tokenizer at Daily Mission 4.
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertEncoder from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertEncoder from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertEncoder: ['cls.predictions.bias',

Search corpus: KorQuAD validation context

In [ ]:
from datasets import load_dataset
dataset = load_dataset("squad_kor_v1")
corpus = list(set([example['context'] for example in dataset['train']]))
len(corpus)

Dataset squad_kor_v1 downloaded and prepared to /root/.cache/huggingface/datasets/squad_kor_v1/squad_kor_v1/1.0.0/31982418accc53b059af090befa81e68880acc667ca5405d30ce6fa7910950a7. Subsequent calls will reuse this data.


9606

In [ ]:
search_corpus = list(set([example['context'] for example in dataset['validation']]))
len(search_corpus)

960

Passage encoder를 활용하여 passage dense embedding 생성

In [ ]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm.auto import tqdm, trange

eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(search_corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
p_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    p_embs.extend(outputs)

In [ ]:
import numpy as np
p_embs = np.array(p_embs)
p_embs.shape  # (num_passage, emb_dim)

(960, 768)

Question encoder를 활용해여 question dense embedding 생성

In [ ]:
np.random.seed(1)

sample_idx = np.random.choice(range(len(dataset['validation'])), 5)
query = dataset['validation'][sample_idx]['question']
ground_truth = dataset['validation'][sample_idx]['context']

query

['대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은?',
 '금강산의 겨울 이름은?',
 '유관순 열사는 당시 어떤 종교를 믿고 있었는가?',
 '1997년 10월 23일, 국회 본회의 대표 연설에서 전두환, 노태우 전 대통령에 대한 사면을 촉구한 새정치 국민회의 의원은?',
 '셰르징거가 찾아왔다가 우연히 푸시캣 돌스에 영입된 곳은?']

In [ ]:
valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

torch.cuda.empty_cache()

q_embs.shape  # (num_query, emb_dim)

(5, 768)

## GPU를 활용하여 passage retrieval 수행하기

GPU에서 exhaustive search 수행

In [ ]:
if torch.cuda.is_available():
  p_embs_cuda = torch.Tensor(p_embs).to('cuda')
  q_embs_cuda = torch.Tensor(q_embs).to('cuda')

In [ ]:
import time
start_time = time.time()

dot_prod_scores = torch.matmul(q_embs_cuda, torch.transpose(p_embs_cuda, 0, 1))

rank = torch.argsort(dot_prod_scores, dim=1, descending=True).squeeze()
print(rank)

print("--- %s seconds ---" % (time.time() - start_time))

tensor([[212, 287, 510,  ..., 145, 809, 181],
        [329, 113, 893,  ..., 145, 351, 181],
        [513, 446, 462,  ..., 448, 452, 181],
        [489, 843, 328,  ..., 785, 452, 181],
        [377, 176, 761,  ..., 512, 216, 181]], device='cuda:0')
--- 0.02081918716430664 seconds ---


In [ ]:
k = 5 

for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  r = rank[i]
  for j in range(k):
    print("Top-%d passage with score %.4f" % (j+1, dot_prod_scores[i][r[j]]))
    print(search_corpus[r[j]])
  print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

Top-1 passage with score 190.1612
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 

## FAISS를 활용하여 CPU에서 passage retrieval 수행하기




FAISS SQ8, IVF 를 활용해서 cpu에서 passage retrieval 실습해보기

In [ ]:
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = p_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(p_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)


In [ ]:
# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(p_embs)
indexer.add(p_embs)

In [ ]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.001992940902709961 seconds ---


In [ ]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

=======[Distance]=======
[[ 9.375433  16.979416  18.334278  19.029072  19.30021  ]
 [20.489332  27.797113  30.972485  32.143345  35.350586 ]
 [20.731392  20.814318  21.522211  22.416315  22.774076 ]
 [ 5.8145723  6.947813   7.8851266  7.8851266  8.185651 ]
 [21.518826  21.916882  22.212915  22.484571  23.440453 ]]


=======[Index of Top-5 Passages]=======
[[212 956 221 952 813]
 [216  89 767 713 647]
 [252 426 565 555  14]
 [489 153 709 360 857]
 [102 396 383 288 743]]


In [ ]:
for i, q in enumerate(query[:1]):
  print("[Search query]\n", q, "\n")
  print("[Ground truth passage]")
  print(ground_truth[i], "\n")

  d = D[i]
  i = I[i]
  for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(search_corpus[i[j]])
  print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

[Ground truth passage]
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의 정확한 위치는 제3장군묘역 우측능선에 위치하고 있으며 단독 묘역이다. 

Top-1 passage with distance 9.3754
2015년 11월 10일 건강검진 차 서울대학교 병원을 찾아 17일까지 입원한 뒤 퇴원했다. 그러다, 이틀 뒤인 19일 고열과 혈액감염 의심 증세로 서울대학교 병원에 다시 입원한 후, 11월 21일 오후에 증세가 급격히 악화됨에 따라 중환자실로 옮겨졌다. 상태가 전혀 호전되지 않던 김영삼은 결국 2015년 11월 22일 오전 0시 21분 32초에 병마를 물리치지 못하고 혈액 감염 의심으로 치료를 받던 중 향년 89세의 일기로 서거하였다. 사망에 이른 직접적인 원인은 허약한 전신 상태에 패혈증과 급성 심부전이 겹쳐 일어난 것으로 판단되었다. 장례는 대한민국 최초로 5일간 국가장으로 치뤄졌다. 이는 국장과 국민장이 통합된 이후 처음 치뤄지는 국가장이다. 이어 11월 26일 국회의사당에서 영결식이 있었고 국립서울현충원에 안장되었다. 묘소의

## [과제 2] Wikipedia documents에 대해 FAISS retrieval 실습해보기

In [ ]:
# 3. 미리 올려둔 wiki docs 다운로드
!wget --load-cookies /tmp/cookies.txt "https://docs.google.com/uc?export=download&confirm=$(wget --quiet --save-cookies /tmp/cookies.txt --keep-session-cookies --no-check-certificate 'https://docs.google.com/uc?export=download&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF' -O- | sed -rn 's/.*confirm=([0-9A-Za-z_]+).*/\1\n/p')&id=1eD57pCCvhiXLrHBb1j07QX1VQQDogejF" -O wikipedia_documents.json && rm -rf /tmp/cookies.txt

In [ ]:
# First load wikipedia dump
import json

# TODO: Write your own path and load json file
dump_path = "./wikipedia_documents.json"
with open(dump_path, 'r', encoding='utf-8') as f:
    wiki = json.load(f)

In [ ]:
corpus = [dic['text'] for dic in wiki.values()]
len(corpus)

60613

In [ ]:
corpus[0]

'이 문서는 나라 목록이며, 전 세계 206개 나라의 각 현황과 주권 승인 정보를 개요 형태로 나열하고 있다.\n\n이 목록은 명료화를 위해 두 부분으로 나뉘어 있다.\n\n# 첫 번째 부분은 바티칸 시국과 팔레스타인을 포함하여 유엔 등 국제 기구에 가입되어 국제적인 승인을 널리 받았다고 여기는 195개 나라를 나열하고 있다.\n# 두 번째 부분은 일부 지역의 주권을 사실상 (데 팍토) 행사하고 있지만, 아직 국제적인 승인을 널리 받지 않았다고 여기는 11개 나라를 나열하고 있다.\n\n두 목록은 모두 가나다 순이다.\n\n일부 국가의 경우 국가로서의 자격에 논쟁의 여부가 있으며, 이 때문에 이러한 목록을 엮는 것은 매우 어렵고 논란이 생길 수 있는 과정이다. 이 목록을 구성하고 있는 국가를 선정하는 기준에 대한 정보는 "포함 기준" 단락을 통해 설명하였다. 나라에 대한 일반적인 정보는 "국가" 문서에서 설명하고 있다.'

In [ ]:
from torch.utils.data import (DataLoader, RandomSampler, TensorDataset, SequentialSampler)
from tqdm.auto import tqdm, trange

eval_batch_size = 8

# Construt dataloader
valid_p_seqs = tokenizer(corpus, padding="max_length", truncation=True, return_tensors='pt')
valid_dataset = TensorDataset(valid_p_seqs['input_ids'], valid_p_seqs['attention_mask'], valid_p_seqs['token_type_ids'])
valid_sampler = SequentialSampler(valid_dataset)
valid_dataloader = DataLoader(valid_dataset, sampler=valid_sampler, batch_size=eval_batch_size)

# Inference using the passage encoder to get dense embeddeings
wiki_embs = []

with torch.no_grad():

  epoch_iterator = tqdm(valid_dataloader, desc="Iteration", position=0, leave=True)
  p_encoder.eval()

  for _, batch in enumerate(epoch_iterator):
    batch = tuple(t.cuda() for t in batch)

    p_inputs = {'input_ids': batch[0],
                'attention_mask': batch[1],
                'token_type_ids': batch[2]
                }
        
    outputs = p_encoder(**p_inputs).to('cpu').numpy()
    wiki_embs.extend(outputs)

import numpy as np
wiki_embs = np.array(wiki_embs)
wiki_embs.shape  # (num_passage, emb_dim)

(60613, 768)

In [ ]:
# TODO: build faiss indexer using your own parameters
import faiss

num_clusters = 16
niter = 5
k = 5

# 1. Clustering
emb_dim = wiki_embs.shape[-1]
index_flat = faiss.IndexFlatL2(emb_dim)

clus = faiss.Clustering(emb_dim, num_clusters)
clus.verbose = True
clus.niter = niter
clus.train(wiki_embs, index_flat)
centroids = faiss.vector_float_to_array(clus.centroids)
centroids = centroids.reshape(num_clusters, emb_dim)

quantizer = faiss.IndexFlatL2(emb_dim)
quantizer.add(centroids)

# 2. SQ8 + IVF indexer (IndexIVFScalarQuantizer)
indexer = faiss.IndexIVFScalarQuantizer(quantizer, quantizer.d, quantizer.ntotal, faiss.METRIC_L2)
indexer.train(wiki_embs)
indexer.add(wiki_embs)

In [ ]:
# TODO: transform query to dense vector
query = '대통령을 포함한 미국의 행정부 견제권을 갖는 국가 기관은?'

valid_q_seqs = tokenizer(query, padding="max_length", truncation=True, return_tensors='pt').to('cuda')

with torch.no_grad():
  q_encoder.eval()
  q_embs = q_encoder(**valid_q_seqs).to('cpu').numpy()

q_embs.shape  # (num_query, emb_dim)

(1, 768)

In [ ]:
# 3. Search using indexer

start_time = time.time()
D, I = indexer.search(q_embs, k)
print("--- %s seconds ---" % (time.time() - start_time))

--- 0.008892059326171875 seconds ---


In [ ]:
print('=======[Distance]=======')
print(D)
print('\n')
print('=======[Index of Top-5 Passages]=======')
print(I)

=======[Distance]=======
[[ 7.8849955 12.298415  12.337481  12.931769  12.988866 ]]


=======[Index of Top-5 Passages]=======
[[36861 13769 23900 37084 18293]]


In [ ]:
print("[Search query]\n", q, "\n")
# 주의: 우리는 wiki 문서 전체에서 임의의 질문을 물어보는 것이므로 ground truth 가 존재하지 않습니다

d = D[0]
i = I[0]
for j in range(k):
    print("Top-%d passage with distance %.4f" % (j+1, d[j]))
    print(corpus[i[j]])
    print('\n')

[Search query]
 대한민국 제14대 대통령으로 향년 89세를 일기로 서거한 김영삼 대통령의 묘소가 있는 곳은? 

Top-1 passage with distance 7.8850
대통령 지시(Presidential directive)는 미국 대통령의 국가안보 관련 행정명령이다. NSC의 조언과 동의(Advice and consent)가 필요하다. 국가안보 대통령 지시가 보다 뜻이 명확하다.

역대 대통령들은 다양한 용어를 사용했다. 케네디 대통령은 국가안보실행메모(NSAM, National Security Action Memorandums), 닉슨과 포드 대통령은 국가안보결정메모(NSDM, National Security Decision Memorandums), 클린턴 대통령은 대통령결정지시, 조지 부시 대통령은 국가안보대통령지시, 오바마 대통령은 대통령정책지시라고 부른다.

미국의 대통령 지시는 비밀명령으로 내려지기도 하는데, 이는 수십년이 지나면 비밀해제되어 일반에 공개된다. 미국 육군 정보와 보안 사령부와 같이, 군대에 특정 부대를 창설하는 경우, 대통령 지시만으로 창설되곤 한다.


Top-2 passage with distance 12.2984
캐나다의 정치는 입헌군주제의 연방제로 강한 민주주의 전통을 가진 의회 정부를 포함하고 있다. 헌법에 해당하는 것은 대영 북아메리카 조약, 웨스트민스터 조례 등의 수정에 의하며, 성문화되어 있지 않다. 원수는 총독이다. 실제의 정치는 입법부에 대하여 책임을 지는 의원내각제에 의해서 행하여지며, 다수당의 당수가 수상이 된다. 연방정부는 국방, 외교, 무역, 금융 등을 통괄하며, 교육·노동 등은 주(州)정부의 권한에 속한다.캐나다의 외교는 영국 연방의 일원으로서 영국과의 관계를 유지하며, 북대서양 조약 기구(NATO)에 가맹하여 군사적, 경제적으로는 미국과 긴밀한 관계에 있다. 미주 기구(美洲機構)에는 1990년에 가맹하였으며 , 국제연합(UN)의 조정적인 활동을 통하여 주체성 유지에 노력하고 있다.

영국